In [17]:
import scrapy
import re
import json
import logging
import pandas as pd
import selenium
import time
import sys
import linecache
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import pymongo
import datetime
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from scrapy.crawler import CrawlerProcess
from time import sleep
from selenium.webdriver.common.keys import Keys
from time import sleep 
from pymongo import MongoClient 
from selenium.webdriver.common.by import By

# YOGYA Scraping

In [18]:
# Print exception
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [30]:
# Scraping the page
def scrape_detail_page(driver,category,collection):
    time.sleep(5)
    # scraping page
    path = driver.find_elements_by_css_selector('.item.product.product-item')
    result = pd.DataFrame()
    #print(len(path))
    for x in range(len(path)):
        element = path[x]
        element.location_once_scrolled_into_view
        error = 0
        while(error<3):
            try:
                sku = path[x].find_element_by_css_selector('.price-box.price-final_price').get_attribute('data-product-id')
                brand = re.sub("  ","",re.sub(" \n","",path[x].find_element_by_css_selector('.product.name.product-brand-name').text))
                name = path[x].find_element_by_css_selector('.product.name.product-item-name').text
                origin_price = path[x].find_elements_by_css_selector('.price')[0].text
                try:
                    discounted_price = path[x].find_elements_by_css_selector('.price')[1].text
                except:
                    discounted_price = origin_price
                image = path[x].find_element_by_css_selector('div > div.product_image > a > span > span > img').get_attribute('data-original')
                
                break
            except:
                PrintException()
                error = error+1
                time.sleep(2)
        # Return result
        date = datetime.datetime.now().strftime("%Y-%m-%d")
        new_result = pd.DataFrame([{'sku':sku,'name':name, 'origin_price':origin_price, 'discounted_price':discounted_price, 'image':image,'category':category,'scraping_date':date}])
        result = result.append(new_result)
        #print(new_result)
            
    # save to mongodb
    try:
        collection.insert_many(result.to_dict('records'))
    except:
        PrintException()
        pass

    #driver.close()
    return result

In [20]:
def scrape_url2(url,category,collection):
        
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    # database
    db = conn.yogya
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]
    
    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    driver = webdriver.Firefox()
    driver.get(url)

    # Start Scraping
    result = pd.DataFrame()
    
    while(True):
        # Go to catalogue page
        result = result.append(scrape_detail_page(driver,category,collection))

        # Go to the next page
        try:
            check_next = driver.find_element_by_css_selector('#maincontent > div.columns > div.column.main > div:nth-child(5) > div.pages > ul > li.item.pages-item-next > a')
            print("Go to next page: " + check_next.get_attribute('href'))
            check_next.click()
        except:
            PrintException()
            break
    
    driver.close()
    return result


In [29]:
# MAIN
yogya_category = pd.read_csv('..\\..\\yogya_category.csv',sep=",")
result = pd.DataFrame()
for x in range(1,len(yogya_category)):
    category = yogya_category.iloc[x]
    print(category['category'])
    
    url = category['url']+"?product_list_limit=30"
    collection = category['collection']
    
    result = scrape_url2(url,category['category'],collection)
    print(result.head())

tops
Connected successfully!!!
  category discounted_price  \
0     tops    Rp 129.000,00   

                                               image           name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  Wo Entire Top   

    origin_price scraping_date    sku  
0  Rp 129.000,00    2019-05-27  52767  
  category discounted_price  \
0     tops    Rp 116.000,00   

                                               image               name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  Wo Eleanor Blouse   

    origin_price scraping_date    sku  
0  Rp 145.000,00    2019-05-27  52739  
  category discounted_price  \
0     tops       Rp 119.200   

                                               image             name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  Wo Danai Blouse   

  origin_price scraping_date    sku  
0   Rp 149.000    2019-05-27  52737  
  category discounted_price  \
0     tops       Rp 116.000   

                                        

  category discounted_price  \
0     tops    Rp 100.000,00   

                                               image              name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Syarani Blouse   

    origin_price scraping_date    sku  
0  Rp 100.000,00    2019-05-27  47121  
  category discounted_price  \
0     tops        Rp 97.000   

                                               image           name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Ekin Blouse   

  origin_price scraping_date    sku  
0    Rp 97.000    2019-05-27  47118  
  category discounted_price  \
0     tops        Rp 97.000   

                                               image            name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Aynur Blouse   

  origin_price scraping_date    sku  
0    Rp 97.000    2019-05-27  47117  
  category discounted_price  \
0     tops        Rp 97.000   

                                               image           name  \
0  https

  category discounted_price  \
0     tops       Rp 100.000   

                                               image              name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Trifena Blouse   

  origin_price scraping_date    sku  
0   Rp 100.000    2019-05-27  46914  
  category discounted_price  \
0     tops       Rp 109.000   

                                               image              name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Devinna Blouse   

  origin_price scraping_date    sku  
0   Rp 109.000    2019-05-27  46912  
Go to next page: https://www.yogyaonline.co.id/women/women-apparel/women-apparel-tops.html?p=3&product_list_limit=30
  category discounted_price  \
0     tops       Rp 109.000   

                                               image            name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Winny Blouse   

  origin_price scraping_date    sku  
0   Rp 109.000    2019-05-27  46910  
  category discounted_p

  category discounted_price  \
0     tops       Rp 103.000   

                                               image            name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Evien Blouse   

  origin_price scraping_date    sku  
0   Rp 103.000    2019-05-27  46809  
  category discounted_price  \
0     tops        Rp 95.000   

                                               image             name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Damiah Blouse   

  origin_price scraping_date    sku  
0    Rp 95.000    2019-05-27  46808  
Go to next page: https://www.yogyaonline.co.id/women/women-apparel/women-apparel-tops.html?p=4&product_list_limit=30
  category discounted_price  \
0     tops        Rp 93.000   

                                               image         name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Irena Top   

  origin_price scraping_date    sku  
0    Rp 93.000    2019-05-27  46807  
  category discounted_price  \
0   

  category discounted_price  \
0     tops       Rp 120.000   

                                               image            name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Pirca Blouse   

  origin_price scraping_date    sku  
0   Rp 120.000    2019-05-27  46544  
  category discounted_price  \
0     tops       Rp 137.000   

                                               image           name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Pime Blouse   

  origin_price scraping_date    sku  
0   Rp 137.000    2019-05-27  46542  
  category discounted_price  \
0     tops       Rp 297.000   

                                               image            name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Nadine Dress   

  origin_price scraping_date    sku  
0   Rp 297.000    2019-05-27  46541  
Go to next page: https://www.yogyaonline.co.id/women/women-apparel/women-apparel-tops.html?p=5&product_list_limit=30
  category discounted_price  \
0 

...

  category discounted_price  \
0        -       Rp 159.200   

                                               image           name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Ashly Mules   

  origin_price scraping_date    sku  
0   Rp 199.000    2019-05-27  39321  
  category discounted_price  \
0        -       Rp 159.200   

                                               image  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...   

                       name origin_price scraping_date    sku  
0  W Amika Polka Flat Shoes   Rp 199.000    2019-05-27  39121  
  category discounted_price  \
0        -       Rp 159.200   

                                               image                 name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Astrid Flat Shoes   

  origin_price scraping_date    sku  
0   Rp 199.000    2019-05-27  39120  
  category discounted_price  \
0        -       Rp 151.200   

                                               image     

EXCEPTION IN (<ipython-input-28-25e13a8d7b77>, LINE 14 "sku = path[x].find_element_by_css_selector('.price-box.price-final_price').get_attribute('data-product-id')"): Message: Unable to locate element: .price-box.price-final_price

  category discounted_price  \
0        -       Rp 217.000   

                                               image                   name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Norvid Theona Shoes   

  origin_price scraping_date    sku  
0   Rp 217.000    2019-05-27  12224  
  category discounted_price  \
0        -       Rp 180.000   

                                               image                     name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Bellita Linda Sandals   

  origin_price scraping_date    sku  
0   Rp 225.000    2019-05-27  12221  
  category discounted_price  \
0        -       Rp 194.400   

                                               image                     name  \
0  https://cdn.yogyaonli

  category discounted_price  \
0        -       Rp 109.900   

                                               image  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...   

                                        name origin_price scraping_date    sku  
0  Happy Duck 1710ww Stelan Batik Pr Mix 449   Rp 109.900    2019-05-27  33868  
  category discounted_price  \
0        -        Rp 19.900   

                                               image  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...   

                               name origin_price scraping_date    sku  
0  Nova Cln Pjg Kaki Cpkwo Warna 06    Rp 19.900    2019-05-27  33867  
  category discounted_price  \
0        -        Rp 69.900   

                                               image  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...   

                                     name origin_price scraping_date    sku  
0  Happy Duck 1710capcpjcars Cpj Cars 464    Rp 69.900    2019-05-27  33866  
  category

  category discounted_price  \
0        -        Rp 55.900   

                                               image                    name  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  Super107 1705kmjtpdmix   

  origin_price scraping_date    sku  
0    Rp 55.900    2019-05-27  32817  
  category discounted_price  \
0        -        Rp 65.900   

                                               image  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...   

                                           name origin_price scraping_date  \
0  Nzbear 1711bepyd2533 Pym Tpd Pr Happy Zo 457    Rp 65.900    2019-05-27   

     sku  
0  32816  
  category discounted_price  \
0        -        Rp 55.900   

                                               image  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...   

                                  name origin_price scraping_date    sku  
0  Super107 1704km8210ss Kmjusa8210 Ss    Rp 55.900    2019-05-27  32815  
  category discount

  category discounted_price  \
0        -       Rp 159.900   

                                               image       name origin_price  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...  W Mem Dot   Rp 159.900   

  scraping_date    sku  
0    2019-05-27  32295  
  category discounted_price  \
0        -       Rp 159.900   

                                               image  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...   

                              name origin_price scraping_date    sku  
0  W Mem Pink Flower (dark Purple)   Rp 159.900    2019-05-27  32294  
  category discounted_price  \
0        -       Rp 159.900   

                                               image  \
0  https://cdn.yogyaonline.co.id/media/catalog/pr...   

                       name origin_price scraping_date    sku  
0  W Mem New Flamingo (red)   Rp 159.900    2019-05-27  32293  
EXCEPTION IN (<ipython-input-20-9643ddfa270b>, LINE 28 "check_next = driver.find_element_by_css_selector